In [ ]:
import pandas as pd
import numpy as np

# Дисциплины

In [ ]:
df_json = pd.read_json('InitialData.json')

In [ ]:
df_json.info()

In [ ]:
# Замена null на NumPy.NaN
df_rpd = df_json.copy()
columns = df_json.columns
for column in columns:
    df_rpd[column] = df_rpd.apply(lambda x: x[column] if x[column] != 'null' else np.nan, axis=1)
    
# Создание df содрежашего идентификатор РПД и кол-во вхождений этого идентификатора (по сути количество семестров изучения дисциплины) в датафрейм. 
df_count_semester = df_rpd.groupby('КодРПД').count().sort_values(by='КодРПД_Т', ascending = False)[['КодРПД_Т']]
# Сортировка позволяющая в последствии удалять все старшие отностиельно начального семестра изучения дисциплины.
df_rpd.sort_values(by=['КодРПД', 'Семестр'], inplace = True)
# Удаление дубликатов РПД (появление дубликатов связано с добавлением в выгрузку поля семестр, которое по правилам реляционной алгебры для дисциплин изучающихся в нескольких семестрах добавляло несколько записей. Количество этих доблирующихся записей определяется количесвтом семестров изучения данной дисциплины)
df_rpd.drop_duplicates(subset = 'КодРПД', keep = 'first', inplace = True)
# Добавление нового поля содержащего количество семестров изучения дисциплины.
df_rpd['Кол-воСеместров'] = df_rpd.apply(lambda x: df_count_semester.loc[x['КодРПД']][0], axis=1)
# Добавление нового поля содержащего последний семестр изучения дисциплины.
df_rpd['ПоследнийСеместр'] = df_rpd.apply(lambda x: x['Семестр'] + x['Кол-воСеместров'] - 1, axis=1)

# Добавление поля с ссылкой на РПД
df_rpd['link'] = df_rpd.apply(lambda x: 'https://edu.donstu.ru/RPD/RPDPrinter.aspx?id=' + str(x['КодРПД']), axis=1)

In [ ]:
df_rpd['rp21_len'] = df_rpd.apply(lambda x: len(str(x['rp21']).strip()), axis=1)
df_rpd['rp1_len'] = df_rpd.apply(lambda x: len(str(x['rp1']).strip()), axis=1)
df_rpd['rp22_len'] = df_rpd.apply(lambda x: len(str(x['rp22']).strip()), axis=1)
df_rpd['rp23_len'] = df_rpd.apply(lambda x: len(str(x['rp23']).strip()), axis=1)
df_rpd['rp52_len'] = df_rpd.apply(lambda x: len(str(x['rp52']).strip()), axis=1)
df_rpd['rpCQ_len'] = df_rpd.apply(lambda x: len(str(x['rpCQ']).strip()), axis=1)
df_rpd['rpMTOD_len'] = df_rpd.apply(lambda x: len(str(x['rpMTOD']).strip()), axis=1)
df_rpd['rpSoftware_len'] = df_rpd.apply(lambda x: len(str(x['rpSoftware']).strip()), axis=1)
df_rpd['rpFund_len'] = df_rpd.apply(lambda x: len(str(x['rpFund']).strip()), axis=1)
df_rpd['rpGuidelines_len'] = df_rpd.apply(lambda x: len(str(x['rpGuidelines']).strip()), axis=1)

In [ ]:
df_rpd[['rp21_len', 'rp1_len', 'rp22_len', 'rp23_len', 'rp52_len', 'rpCQ_len', 'rpMTOD_len', 'rpSoftware_len', 'rpFund_len', 'rpGuidelines_len']].describe()

In [ ]:
df_rpd['text_RPD'] = df_rpd.apply(lambda x: ' '.join(str(str(x['КодРПД']) + ' ' + str(x['rp1']) + str(x['rpCQ'])).replace('\n', ' ').replace('\t+', ' ').replace('\t-', ' ').replace('\t', ' ').replace('\r', ' ').strip().split()), axis=1)

df_rpd['text_RPD_len'] = df_rpd.apply(lambda x: len(str(x['text_RPD']).strip()), axis=1)
df_rpd.text_RPD_len.describe()

# Связи

In [ ]:
df_relationships_all = pd.read_excel('all_relation_v1.xlsx')

In [ ]:
# Выделение датафреима содержашего только пререквизиты
df_prerequisites = df_relationships_all[df_relationships_all['type'] == 0]
# Выделение датафреима содержащего только постреквизиты
df_postrequisites = df_relationships_all[df_relationships_all['type'] == 1]

In [ ]:
# Вычленение связей текущих дисциплин (на текущий учебный год) по типу ПРЕРЕКВИЗИТ (type 0)
df_prerequisites['available'] = df_prerequisites.apply(lambda x: True if df_rpd['КодРПД'].isin([x['rpdId']]).any() and df_rpd['КодРПД'].isin([x['rpdId1']]).any() else False, axis=1)
# Вычленение связей текущих дисциплин (на текущий учебный год) по типу ПОСТРЕКВИЗИТ (type 1)
df_postrequisites['available'] = df_postrequisites.apply(lambda x: True if df_rpd['КодРПД'].isin([x['rpdId']]).any() and df_rpd['КодРПД'].isin([x['rpdId1']]).any() else False, axis=1)

In [ ]:
print(df_prerequisites[df_prerequisites['available'] == True].count()[0])
df_prerequisites = df_prerequisites[df_prerequisites['available'] == True]
df_prerequisites.drop_duplicates(subset = ['rpdId','rpdId1'], keep = 'first', inplace = True)
print(df_prerequisites.count()[0])

In [ ]:
print(df_postrequisites[df_postrequisites['available'] == True].count()[0])
df_postrequisites = df_postrequisites[df_postrequisites['available'] == True]
df_postrequisites.drop_duplicates(subset = ['rpdId','rpdId1'], keep = 'first', inplace = True)
print(df_postrequisites.count()[0])

In [ ]:
from numba import jit

@jit(nopython=True, parallel=True)
def compare(_prereq, _postreq):
    count = 0
    for id_1, pre in _prereq:
        for id_2, post in _postreq:
            if (id_1 == post) and (pre == id_2):
                count += 1       
    return count

In [ ]:
id_subject_pre, id_prerequisit = np.array(df_prerequisites.rpdId.tolist()), np.array(df_prerequisites.rpdId1.tolist())
id_subject_post, id_postrequisit = np.array(df_postrequisites.rpdId.tolist()), np.array(df_postrequisites.rpdId1.tolist())

prereq = list(zip(id_subject_pre, id_prerequisit))
postreq = list(zip(id_subject_post, id_postrequisit))

In [ ]:
%%time
count_conform_relation = compare(prereq, postreq)
print('# Total conform relationship: ' + str(count_conform_relation))
print('# Percent conform relationship among the prerequisites: ' + str(count_conform_relation/len(prereq) * 100))
print('# Percent conform relationship among the postrequisites: ' + str(count_conform_relation/len(postreq) * 100))

# Экспорт данных

In [ ]:
df_prerequisites[['rpdId', 'rpdId1']].to_csv('prerequisites_to_neo4j.csv', header = ['id_postrequisit', 'id_prerequisit'], encoding = 'utf-8', index = False)
df_postrequisites[['rpdId', 'rpdId1']].to_csv('postrequisites_to_neo4j.csv', header = ['id_prerequisit', 'id_postrequisit'], encoding = 'utf-8', index = False)
df_rpd[['КодРПД', 'УчГод', 'Курс', 'Семестр', 'Кол-воСеместров', 'ПоследнийСеместр', 'Предмет', 'Авторы', 'РУП', 'ТитулРУП', 'ЗЕТфакт', 'ШифрООП', 'НазваниеООП', 'ТипГОСа', 'ДатаУтверждения', 'АббрКаф', 'ЦиклКод', 'normZach', 'protocolDate', 'link']].to_csv('subjects_to_neo4j.csv', header = ['id_RPD', 'academicYear', 'course', 'firstSemester', 'numberSemesters', 'lastSemester', 'titleSubject', 'author', 'RUP', 'OOP', 'teachingLoad', 'codeOOP', 'titleOOP', 'typeGOSa', 'approvalDate', 'abbrDepartment', 'cycleCode', 'normZach', 'protocolDate', 'link'], encoding = 'utf-8', index = False)